## Feature selection experiment

In [4]:
import matplotlib.pyplot as plt
import itertools
import json
import pandas as pd
import re

resampple = r"sample (\d+)"
is_sampled = r"timeout (true|false)"




def load_mutable_csv(f, rename=""):
    f = f.replace(".csv", "")
    
    a = pd.read_csv(f"csv/{f}.csv",index_col = 'id')
    
    if rename:
        a.rename(columns = {'mutable_count': rename}, inplace = True)
        
        snapthots = [ "_t30.snapshot", "_t5.snapshot"]
        # Set the sample ratio then
        if len(a[a[rename] > 0]) == 0:
            # trying with the snapshots
            
            for c in snapthots:
                try:
                    print(f"Trying csv/{f}{c}.csv")
                    np = pd.read_csv(f"csv/{f}{c}.csv",index_col = 'id')
                    if len(np[np['mutable_count'] > 0]) > 0:
                        print(f"Taking {c} for {f}" )
                        a = np
                        a.rename(columns = {'mutable_count': rename}, inplace = True)
                except Exception as e:
                    print(e)
                    pass
            if len(a[a[rename] > 0]) == 0:
                print(">>>>> Zero count", rename, f)
        else:
            pass
            #print(rename)
        
    
    #display(a)
    a[rename] = a[rename] > 0
    return a



In [5]:

ratios = [
    
]



files = [('peep.i32.operator_or_neg1', 'i32.operator_or_neg1.csv'), 
         ('peep.i32.operator_or_neg_one1', 'i32.operator_or_neg_one1.csv'), 
         ('peep.i32.operator_or_commutative', 'i32.operator_or_commutative.csv'), 
         ('peep.i64.operator_or_commutative', 'i64.operator_or_commutative.csv'), 
         ('peep.i32.operator_and_commutative', 'i32.operator_and_commutative.csv'), 
         ('peep.i64.operator_and_commutative', 'i64.operator_and_commutative.csv'), 
         ('peep.select_same_branches', 'select_same_branches.csv'), 
         ('peep.i32.sub_zero', 'i32.sub_zero.csv'), 
         ('peep.i32.mul_1', 'i32.mul_1_t5.snapshot.csv'), 
         ('peep.i64.mul_1', 'i64.mul_1.csv'), 
         ('peep.f32.mul_1', 'f32.mul_1.csv'), 
         ('peep.f64.mul_1', 'f64.mul_1.csv'), 
         ('peep.i32.add_0', 'i32.add_0.csv'), 
         ('peep.i64.add_0', 'i64.add_0.csv'), 
         ('peep.f32.add_0', 'f32.add_0.csv'), 
         ('peep.f64.add_0', 'f64.add_0.csv'), 
         ('peep.i32.xor_0', 'i32.xor_0.csv'), 
         ('peep.i64.xor_0', 'i64.xor_0.csv'), 
         ('peep.i32.eq_0', 'i32.eq_0.csv'),
         ('peep.i64.eq_0', 'i64.eq_0.csv'), 
         ('peep.i32.shl_by_0', 'i32.shl_by_0.csv'), 
         ('peep.i64.shl_by_0', 'i64.shl_by_0.csv'), 
         ('peep.i32.shr_u_by_0', 'i32.shr_u_by_0.csv'), 
         ('peep.i64.shr_u_by_0', 'i64.shr_u_by_0.csv'), 
         ('peep.i32.shr_s_by_0', 'i32.shr_s_by_0.csv'), 
         ('peep.i64.shr_s_by_0', 'i64.shr_s_by_0.csv'), 
         ('peep.i64.operator_or_neg1', 'i64.operator_or_neg1.csv'), 
         ('peep.i64.sub_zero', 'i64.sub_zero.csv'), 
         ('peep.i32.add-commutes', 'i32.add-commutes.csv'), 
         ('peep.i64.add-commutes', 'i64.add-commutes.csv'), 
         ('peep.f32.add-commutes', 'f32.add-commutes.csv'), 
         ('peep.f64.add-commutes', 'f64.add-commutes.csv'), 
         ('peep.i32.mul-commutes', 'i32.mul-commutes.csv'), 
         ('peep.i64.mul-commutes', 'i64.mul-commutes.csv'), 
         ('peep.f32.mul-commutes', 'f32.mul-commutes.csv'), 
         ('peep.f64.mul-commutes', 'f64.mul-commutes.csv'), 
         ('peep.i32.and-commutes', 'i32.and-commutes.csv'), 
         ('peep.i64.and-commutes', 'i64.and-commutes.csv'), 
         ('peep.i32.or-commutes', 'i32.or-commutes.csv'), 
         ('peep.i64.or-commutes', 'i64.or-commutes.csv'), 
         ('peep.i32.xor-commutes', 'i32.xor-commutes.csv'), 
         ('peep.i64.xor-commutes', 'i64.xor-commutes.csv'), 
         ('peep.i32.eq-commutes', 'i32.eq-commutes.csv'), 
         ('peep.i64.eq-commutes', 'i64.eq-commutes.csv'), 
         ('peep.i32.mul-associates', 'i32.mul-associates.csv'), 
         ('peep.i64.mul-associates', 'i64.mul-associates.csv'), 
         ('peep.i32.add-associates', 'i32.add-associates.csv'), 
         ('peep.i32.and-associates', 'i32.and-associates.csv'), 
         ('peep.i32.or-associates', 'i32.or-associates.csv'), 
         ('peep.i64.or-associates', 'i64.or-associates.csv'), 
         ('peep.i32.xor-associates', 'i32.xor-associates.csv'), 
         ('peep.i64.xor-associates', 'i64.xor-associates.csv'), 
         ('peep.i32.eq-associates', 'i32.eq-associates.csv'), 
         ('peep.i64.eq-associates', 'i64.eq-associates.csv'), 
         ('peep.i32.mul-by-2', 'i32.mul-by-2.csv'), 
         ('peep.i64.mul-by-2', 'i64.mul-by-2.csv'), 
         ('peep.i32.mul-by-4', 'i32.mul-by-4.csv'), 
         ('peep.i64.mul-by-4', 'i64.mul-by-4.csv'), 
         ('peep.i32.mul-by-8', 'i32.mul-by-8.csv'), 
         ('peep.i64.mul-by-8', 'i64.mul-by-8.csv'), 
         ('peep.select-invert', 'select-invert.csv'), 
         ('peep.i32.add-x-x', 'i32.add-x-x.csv'), 
         ('peep.i64.add-x-x', 'i64.add-x-x.csv'), 
         ('peep.i32.drop-x', 'i32.drop-x.csv'), 
         ('peep.i64.drop-x', 'i64.drop-x.csv'), 
         ('peep.f32.drop-x', 'f32.drop-x.csv'), 
         ('peep.f64.drop-x', 'f64.drop-x.csv'), 
         ('peep.container-nop-x', 'container-nop-x.csv'), 
         ('peep.container-x-nop', 'container-x-nop.csv'), 
         ('peep.container-drop-i32.rand-x', 'container-drop-i32.rand-x.csv'), 
         ('peep.container-drop-i64.rand-x', 'container-drop-i64.rand-x.csv'), 
         ('peep.container-x-drop-i32.rand', 'container-x-drop-i32.rand.csv'), 
         ('peep.container-x-drop-i64.rand', 'container-x-drop-i64.rand.csv'), 
         ('peep.i32.use_of_global', 'i32.use_of_global.csv'), 
         ('peep.i64.use_of_global', 'i64.use_of_global.csv'), 
         ('peep.f32.use_of_global', 'f32.use_of_global.csv'), 
         ('peep.f64.use_of_global', 'f64.use_of_global.csv'), 
         ('peep.i32.unfold', 'i32.unfold.csv'), 
         ('peep.i64.unfold', 'i64.unfold_t30.csv'), 
         ('peep.replace-with-i32-1', 'replace-with-i32-1.csv'), 
         ('peep.replace-with-i64-1', 'replace-with-i64-1_t30.csv'),
         ('peep.replace-with-f32-1', 'replace-with-f32-1_t30.csv'), 
         ('peep.replace-with-f64-1', 'replace-with-f64-1_t30.csv'), 
         ('peep.replace-with-v128-1', 'replace-with-v128-1_t30.csv'), 
         ('peep.replace-with-i32.rand', 'replace-with-i32.rand_t30.csv'), 
         ('peep.replace-with-i64.rand', 'replace-with-i64.rand.csv'), 
         ('peep.replace-with-f32.rand', 'replace-with-f32.rand.csv'), 
         ('peep.replace-with-f64.rand', 'replace-with-f64.rand.csv'), 
         ('peep.replace-with-ref-null-func', 'replace-with-ref-null-func.csv'), 
         ('peep.replace-with-ref-null-extern', 'replace-with-ref-null-extern.csv'), 
         ('peep.remove-drop', 'remove-drop.csv'), 
         ('peep.remove-nop', 'remove-nop.csv'), 
         ('peep.remove-global.set.0', 'remove-global.set.0.csv'), 
         ('peep.remove-global.set.1', 'remove-global.set.1.csv'), 
         ('peep.remove-elem.drop.0', 'remove-elem.drop.0.csv'), 
         ('peep.remove-elem.drop.1', 'remove-elem.drop.1.csv'), 
         ('peep.remove-data.drop.0', 'remove-data.drop.0.csv'), 
         ('peep.remove-data.drop.1', 'remove-data.drop.1.csv'), 
         ('peep.i32.add-1', 'i32.add-1.csv'), 
         ('peep.i64.add-1', 'i64.add-1.csv'), 
         ('peep.i32.sub-1', 'i32.sub-1.csv'), 
         ('peep.i64.sub-1', 'i64.sub-1.csv'), 
         ('peep.i32.and-1', 'i32.and-1.csv'), 
         ('peep.i64.and-1', 'i64.and-1.csv'), 
         ('peep.i32.or-1', 'i32.or-1.csv'), 
         ('peep.i64.or-1', 'i64.or-1.csv'), 
         ('peep.i32.xor-1', 'i32.xor-1.csv'), 
         ('peep.i64.xor-1', 'i64.xor-1.csv'), 
         ('peep.i32.shl-1', 'i32.shl-1.csv'), 
         ('peep.i64.shl-1', 'i64.shl-1.csv'), 
         ('peep.i32.shr_u-1', 'i32.shr_u-1.csv'), 
         ('peep.i64.shr_u-1', 'i64.shr_u-1.csv'), 
         ('peep.i32.shr_s-1', 'i32.shr_s-1.csv'), 
         ('peep.i64.shr_s-1', 'i64.shr_s-1.csv'), 
         ('code_motion_ifs', 'code_motion_ifs.csv'), 
         ('code_motion_loops', 'code_motion_loops.csv'), 
         ('add_function', 'add_function.csv'), 
         ('add_type', 'add_type.csv'), 
         ('modify_custom_section', 'modify_custom_section_name.csv'), 
         ('modify_custom_section_name', 'modify_custom_section_name.csv'), 
         ('modify_custom_section_data', 'modify_custom_section_data.csv'), 
         ('function_body_unreachable', 'function_body_unreachable.csv'), 
         ('modify_data', 'modify_data.csv'), 
         ('modify_init_exprs', 'modify_init_exprs.csv'), 
         ('remove_exports', 'remove_exports.csv'), 
         ('remove_item_function', 'remove_item_function.csv'), 
         ('remove_item_table', 'remove_item_table.csv'), 
         ('remove_item_memory', 'remove_item_memory.csv'), 
         ('remove_item_global', 'remove_item_global.csv'), 
         ('remove_item_tag', 'remove_item_tag.csv'), 
         ('remove_item_type', 'remove_item_type.csv'), 
         ('remove_item_data', 'remove_item_data.csv'), 
         ('remove_item_element', 'remove_item_element.csv'), 
         ('rename_exports', 'rename_exports.csv'), 
         ('snip_function', 'snip_function.csv')]
alldf = pd.DataFrame()

for n, f in files:
    print(n)
    df = load_mutable_csv(f, n)
    alldf = pd.concat([df, alldf], axis=1)

# Replace 0 by non mutable :|
alldf=alldf.fillna(False)
alldf = alldf.loc[:,~alldf.columns.duplicated()].copy()

A = 1

peep.i32.operator_or_neg1
peep.i32.operator_or_neg_one1
Trying csv/i32.operator_or_neg_one1_t30.snapshot.csv
[Errno 2] No such file or directory: 'csv/i32.operator_or_neg_one1_t30.snapshot.csv'
Trying csv/i32.operator_or_neg_one1_t5.snapshot.csv
>>>>> Zero count peep.i32.operator_or_neg_one1 i32.operator_or_neg_one1
peep.i32.operator_or_commutative
peep.i64.operator_or_commutative
peep.i32.operator_and_commutative
peep.i64.operator_and_commutative
peep.select_same_branches
peep.i32.sub_zero
peep.i32.mul_1
peep.i64.mul_1
peep.f32.mul_1
peep.f64.mul_1
peep.i32.add_0
peep.i64.add_0
peep.f32.add_0
peep.f64.add_0
peep.i32.xor_0
peep.i64.xor_0
peep.i32.eq_0
peep.i64.eq_0
peep.i32.shl_by_0
peep.i64.shl_by_0
peep.i32.shr_u_by_0
peep.i64.shr_u_by_0
peep.i32.shr_s_by_0
peep.i64.shr_s_by_0
peep.i64.operator_or_neg1
Trying csv/i64.operator_or_neg1_t30.snapshot.csv
[Errno 2] No such file or directory: 'csv/i64.operator_or_neg1_t30.snapshot.csv'
Trying csv/i64.operator_or_neg1_t5.snapshot.csv
>>>>> 

peep.i32.add-1
Trying csv/i32.add-1_t30.snapshot.csv
[Errno 2] No such file or directory: 'csv/i32.add-1_t30.snapshot.csv'
Trying csv/i32.add-1_t5.snapshot.csv
>>>>> Zero count peep.i32.add-1 i32.add-1
peep.i64.add-1
Trying csv/i64.add-1_t30.snapshot.csv
[Errno 2] No such file or directory: 'csv/i64.add-1_t30.snapshot.csv'
Trying csv/i64.add-1_t5.snapshot.csv
>>>>> Zero count peep.i64.add-1 i64.add-1
peep.i32.sub-1
Trying csv/i32.sub-1_t30.snapshot.csv
[Errno 2] No such file or directory: 'csv/i32.sub-1_t30.snapshot.csv'
Trying csv/i32.sub-1_t5.snapshot.csv
>>>>> Zero count peep.i32.sub-1 i32.sub-1
peep.i64.sub-1
Trying csv/i64.sub-1_t30.snapshot.csv
[Errno 2] No such file or directory: 'csv/i64.sub-1_t30.snapshot.csv'
Trying csv/i64.sub-1_t5.snapshot.csv
>>>>> Zero count peep.i64.sub-1 i64.sub-1
peep.i32.and-1
Trying csv/i32.and-1_t30.snapshot.csv
[Errno 2] No such file or directory: 'csv/i32.and-1_t30.snapshot.csv'
Trying csv/i32.and-1_t5.snapshot.csv
>>>>> Zero count peep.i32.and-1 

In [6]:
display(alldf)

,num_tags,num_functions,num_globals,num_tables,num_elements,num_data,num_types,num_memory,num_instructions,class_name,...,peep.i64.mul_1,peep.i32.mul_1,peep.i32.sub_zero,peep.select_same_branches,peep.i64.operator_and_commutative,peep.i32.operator_and_commutative,peep.i64.operator_or_commutative,peep.i32.operator_or_commutative,peep.i32.operator_or_neg_one1,peep.i32.operator_or_neg1
id,,,,,,,,,,,,,,,,,,,,,
00008799c04b4f111d28331ef77c387fb2bd6724c9ce88a80f30cd320c044c29.wasm,0,19,3,1,1,0,10,1,1751,none,...,True,False,True,False,True,True,True,True,False,False
000415b2639c960e039299fde2a52e7e9c692c212c1fa6ca4701eec637ff6a30.wasm,0,37,6,0,1,0,10,0,28339,none,...,True,False,True,False,True,True,True,True,False,False
000e01741a75cf7a3d9afc0f4950c220c67188a7c1434bb6699654e00d07c1da.wasm,0,72,27,1,1,0,16,1,14859,none,...,True,False,True,False,True,True,True,True,False,False
00189ff4eeb814ffa662649ba6859c004036580fccc2b83862cda60f5737d5d1.wasm,0,37,0,1,1,0,19,1,5313,none,...,True,False,True,False,True,True,True,True,False,False
0018e8f82de999af365dfc57d0bbe06d5d08eaf791d31f59af0eecd53d363efa.wasm,0,25,0,1,1,0,11,1,2454,none,...,True,True,True,False,True,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffc74ca2c5ca93aefa0a7e7e0f2b73fa04f82ef95f52de4bfe09546d5a884a45.wasm,0,13354,9,0,1,0,190,0,903659,none,...,True,False,True,False,True,True,True,True,False,False
ffd880b33124a740f739dda080d1969280ea4d6c1772f4d4875589fd2e6328c2.wasm,0,73,27,1,1,0,17,1,14615,none,...,True,False,True,False,True,True,True,True,False,False
ffe3c93afa93da108768ccbc6ae0fdf0a493f3177e3164b1ca76ddd5ca0902c1.wasm,0,4,1,1,0,0,6,1,224,none,...,True,False,True,False,True,True,True,True,False,False


In [18]:
import random



features = [('peep.i32.operator_or_neg1', 'i32.operator_or_neg1'), 
         ('peep.i32.operator_or_neg_one1', 'i32.operator_or_neg_one1'), 
         ('peep.i32.operator_or_commutative', 'i32.operator_or_commutative'), 
         ('peep.i64.operator_or_commutative', 'i64.operator_or_commutative'), 
         ('peep.i32.operator_and_commutative', 'i32.operator_and_commutative'), 
         ('peep.i64.operator_and_commutative', 'i64.operator_and_commutative'), 
         ('peep.select_same_branches', 'select_same_branches'), 
         ('peep.i32.sub_zero', 'i32.sub_zero'), 
         ('peep.i32.mul_1', 'i32.mul_1_t5.snapshot'), 
         ('peep.i64.mul_1', 'i64.mul_1'), 
         ('peep.f32.mul_1', 'f32.mul_1'), 
         ('peep.f64.mul_1', 'f64.mul_1'), 
         ('peep.i32.add_0', 'i32.add_0'), 
         ('peep.i64.add_0', 'i64.add_0'), 
         ('peep.f32.add_0', 'f32.add_0'), 
         ('peep.f64.add_0', 'f64.add_0'), 
         ('peep.i32.xor_0', 'i32.xor_0'), 
         ('peep.i64.xor_0', 'i64.xor_0'), 
         ('peep.i32.eq_0', 'i32.eq_0'),
         ('peep.i64.eq_0', 'i64.eq_0'), 
         ('peep.i32.shl_by_0', 'i32.shl_by_0'), 
         ('peep.i64.shl_by_0', 'i64.shl_by_0'), 
         ('peep.i32.shr_u_by_0', 'i32.shr_u_by_0'), 
         ('peep.i64.shr_u_by_0', 'i64.shr_u_by_0'), 
         ('peep.i32.shr_s_by_0', 'i32.shr_s_by_0'), 
         ('peep.i64.shr_s_by_0', 'i64.shr_s_by_0'), 
         ('peep.i64.operator_or_neg1', 'i64.operator_or_neg1'), 
         ('peep.i64.sub_zero', 'i64.sub_zero'), 
         ('peep.i32.add-commutes', 'i32.add-commutes'), 
         ('peep.i64.add-commutes', 'i64.add-commutes'), 
         ('peep.f32.add-commutes', 'f32.add-commutes'), 
         ('peep.f64.add-commutes', 'f64.add-commutes'), 
         ('peep.i32.mul-commutes', 'i32.mul-commutes'), 
         ('peep.i64.mul-commutes', 'i64.mul-commutes'), 
         ('peep.f32.mul-commutes', 'f32.mul-commutes'), 
         ('peep.f64.mul-commutes', 'f64.mul-commutes'), 
         ('peep.i32.and-commutes', 'i32.and-commutes'), 
         ('peep.i64.and-commutes', 'i64.and-commutes'), 
         ('peep.i32.or-commutes', 'i32.or-commutes'), 
         ('peep.i64.or-commutes', 'i64.or-commutes'), 
         ('peep.i32.xor-commutes', 'i32.xor-commutes'), 
         ('peep.i64.xor-commutes', 'i64.xor-commutes'), 
         ('peep.i32.eq-commutes', 'i32.eq-commutes'), 
         ('peep.i64.eq-commutes', 'i64.eq-commutes'), 
         ('peep.i32.mul-associates', 'i32.mul-associates'), 
         ('peep.i64.mul-associates', 'i64.mul-associates'), 
         ('peep.i32.add-associates', 'i32.add-associates'), 
         ('peep.i32.and-associates', 'i32.and-associates'), 
         ('peep.i32.or-associates', 'i32.or-associates'), 
         ('peep.i64.or-associates', 'i64.or-associates'), 
         ('peep.i32.xor-associates', 'i32.xor-associates'), 
         ('peep.i64.xor-associates', 'i64.xor-associates'), 
         ('peep.i32.eq-associates', 'i32.eq-associates'), 
         ('peep.i64.eq-associates', 'i64.eq-associates'), 
         ('peep.i32.mul-by-2', 'i32.mul-by-2'), 
         ('peep.i64.mul-by-2', 'i64.mul-by-2'), 
         ('peep.i32.mul-by-4', 'i32.mul-by-4'), 
         ('peep.i64.mul-by-4', 'i64.mul-by-4'), 
         ('peep.i32.mul-by-8', 'i32.mul-by-8'), 
         ('peep.i64.mul-by-8', 'i64.mul-by-8'), 
         ('peep.select-invert', 'select-invert'), 
         ('peep.i32.add-x-x', 'i32.add-x-x'), 
         ('peep.i64.add-x-x', 'i64.add-x-x'), 
         ('peep.i32.drop-x', 'i32.drop-x'), 
         ('peep.i64.drop-x', 'i64.drop-x'), 
         ('peep.f32.drop-x', 'f32.drop-x'), 
         ('peep.f64.drop-x', 'f64.drop-x'), 
         ('peep.container-nop-x', 'container-nop-x'), 
         ('peep.container-x-nop', 'container-x-nop'), 
         ('peep.container-drop-i32.rand-x', 'container-drop-i32.rand-x'), 
         ('peep.container-drop-i64.rand-x', 'container-drop-i64.rand-x'), 
         ('peep.container-x-drop-i32.rand', 'container-x-drop-i32.rand'), 
         ('peep.container-x-drop-i64.rand', 'container-x-drop-i64.rand'), 
         ('peep.i32.use_of_global', 'i32.use_of_global'), 
         ('peep.i64.use_of_global', 'i64.use_of_global'), 
         ('peep.f32.use_of_global', 'f32.use_of_global'), 
         ('peep.f64.use_of_global', 'f64.use_of_global'), 
         ('peep.i32.unfold', 'i32.unfold'), 
         ('peep.i64.unfold', 'i64.unfold'), 
         ('peep.replace-with-i32-1', 'replace-with-i32-1'), 
         ('peep.replace-with-i64-1', 'replace-with-i64-1'),
         ('peep.replace-with-f32-1', 'replace-with-f32-1_t30'), 
         ('peep.replace-with-f64-1', 'replace-with-f64-1_t30'), 
         ('peep.replace-with-v128-1', 'replace-with-v128-1_t30'), 
         ('peep.replace-with-i32.rand', 'replace-with-i32.rand_t30'), 
         ('peep.replace-with-i64.rand', 'replace-with-i64.rand'), 
         ('peep.replace-with-f32.rand', 'replace-with-f32.rand'), 
         ('peep.replace-with-f64.rand', 'replace-with-f64.rand'), 
         ('peep.replace-with-ref-null-func', 'replace-with-ref-null-func'), 
         ('peep.replace-with-ref-null-extern', 'replace-with-ref-null-extern'), 
         ('peep.remove-drop', 'remove-drop'), 
         ('peep.remove-nop', 'remove-nop'), 
         ('peep.remove-global.set.0', 'remove-global.set.0'), 
         ('peep.remove-global.set.1', 'remove-global.set.1'), 
         ('peep.remove-elem.drop.0', 'remove-elem.drop.0'), 
         ('peep.remove-elem.drop.1', 'remove-elem.drop.1'), 
         ('peep.remove-data.drop.0', 'remove-data.drop.0'), 
         ('peep.remove-data.drop.1', 'remove-data.drop.1'), 
         ('peep.i32.add-1', 'i32.add-1'), 
         ('peep.i64.add-1', 'i64.add-1'), 
         ('peep.i32.sub-1', 'i32.sub-1'), 
         ('peep.i64.sub-1', 'i64.sub-1'), 
         ('peep.i32.and-1', 'i32.and-1'), 
         ('peep.i64.and-1', 'i64.and-1'), 
         ('peep.i32.or-1', 'i32.or-1'), 
         ('peep.i64.or-1', 'i64.or-1'), 
         ('peep.i32.xor-1', 'i32.xor-1'), 
         ('peep.i64.xor-1', 'i64.xor-1'), 
         ('peep.i32.shl-1', 'i32.shl-1'), 
         ('peep.i64.shl-1', 'i64.shl-1'), 
         ('peep.i32.shr_u-1', 'i32.shr_u-1'), 
         ('peep.i64.shr_u-1', 'i64.shr_u-1'), 
         ('peep.i32.shr_s-1', 'i32.shr_s-1'), 
         ('peep.i64.shr_s-1', 'i64.shr_s-1'), 
         ('code_motion_ifs', 'code_motion_ifs'), 
         ('code_motion_loops', 'code_motion_loops'), 
         ('add_function', 'add_function'), 
         ('add_type', 'add_type'), 
         ('modify_custom_section', 'modify_custom_section_name'), 
         ('modify_custom_section_name', 'modify_custom_section_name'), 
         ('modify_custom_section_data', 'modify_custom_section_data'), 
         ('function_body_unreachable', 'function_body_unreachable'), 
         ('modify_data', 'modify_data'), 
         ('modify_init_exprs', 'modify_init_exprs'), 
         ('remove_exports', 'remove_exports'), 
         ('remove_item_function', 'remove_item_function'), 
         ('remove_item_table', 'remove_item_table'), 
         ('remove_item_memory', 'remove_item_memory'), 
         ('remove_item_global', 'remove_item_global'), 
         ('remove_item_tag', 'remove_item_tag'), 
         ('remove_item_type', 'remove_item_type'), 
         ('remove_item_data', 'remove_item_data'), 
         ('remove_item_element', 'remove_item_element'), 
         ('rename_exports', 'rename_exports'), 
         ('snip_function', 'snip_function')]


def all_combinations(features, r, cb):
    
    if len(features) == 0:
        cb(r)
    else:
        l = features[0]
        tail = [x for x in features[1:]]
        all_combinations(tail, r + [l], cb)
        all_combinations(tail, r, cb)
    

def get_uniform_x(count):
    
    r = []
    sets = []
    
    alla_filtered = [r for r in features]
    
    if count > 2**(len(alla_filtered)):
        # Filter ouur, there is not possible to get more than this number
        all_combinations(alla_filtered, [], lambda x: r.append([ y[1] for y in x ]))
        return r
        
    while len(r) < count:
        alla = [r[1] for r in features]
        selected = [] 
        
        for x in alla:
            n = random.random()
            if n <= 1/len(features):
                selected.append(x)
                
        selected = list(sorted(selected))
        if len(selected) > 0 and f"{selected}" not in sets:
            r.append(selected)
            sets.append(f"{selected}")
    
    return r

def get_uniform_x_filter_non_probable(frame, count, th=0.8, force_included_features=[]):
    
    def is_probable(name):
        c = len(frame[frame[name] == True])
        return c/len(frame) >= th
    
    r = []
    sets = []
    alla_filtered = [r for r in features if is_probable(r[0]) or r[0] in force_included_features]
    
    print("Set size", len(alla_filtered))
    
    if count > 2**(len(alla_filtered)):
        # Filter ouur, there is not possible to get more than this number
        print("Setting the count to the max possible")
        all_combinations(alla_filtered, [], lambda x: r.append([ y[1] for y in x ]))
        return r
        
    
    while len(r) < count:
        alla = [r for r in alla_filtered]
        #print(alla)
        selected = [] 
        
        for x in alla:
            n = random.random()
            if n <= 1/len(features):
                selected.append(x)
                
        selected = list(sorted(selected))
        if len(selected) > 0 and f"{selected}" not in sets:
            r.append(selected)
            sets.append(f"{selected}")
    
    return r


def get_x_weight_by_applicability(frame, count, reverse = False):
    
    def weight(name):
        c1 = len(frame[frame[name] == True])
        c2 = len(frame)
        
        return (1 - c1/c2) if (reverse and c1 > 0) else c1/ c2
    
    r = []
    sets = []
    weights = {r[0]: weight(r[0]) for r in features }
    sumweights = sum(r for r in weights.values())
    print(weights)
    
    
    while len(r) < count:
        try:
            alla = [r for r in features]
            #print(alla)
            selected = [] 

            for x in alla:
                n = random.random()
                p_perfeature = weights[x[0]]
                p_weighted = p_perfeature/sumweights
                
                if n <= p_weighted:
                    selected.append(x)

            selected = list(sorted(selected))
            if len(selected) > 0 and f"{selected}" not in sets:
                r.append(selected)
                sets.append(f"{selected}")
        except KeyboardInterrupt:
            break
    return r


def get_all_simple(frame, reverse=False):
    
    def weight(name):
        c1 = len(frame[frame[name] == True])
        c2 = len(frame)
        
        return (1 - c1/c2) if (reverse and c1 > 0) else c1/ c2
    
    r = []
    weights = {r[0]: weight(r[0]) for r in features }

    alla = [r for r in features]
    #print(alla)
    selected = [] 

    for x in alla:
        p_perfeature = weights[x[0]]

        if p_perfeature > 0:
            selected = [x] 

            selected = list(sorted(selected))
            if len(selected) > 0 and f"{selected}":
                r.append(selected)
    return r

In [17]:
def print_to_compile(cfgs, create_script=True, attempts=1000):
    
    if attempts%len(cfgs) != 0:
        raise Exception("All configs should have the same number of trials")
    if create_script:
        print("INPUT=$1")
        print("shift 1")
        print()
        print("cargo build --release --features wasm-mutate/all")
        print(f"RUST_LOG=debug ./target/release/analyzer --dbconn datas/cf_all mutate -s 100 -e --attempts {attempts}  --input $INPUT --oracle $@ 2>all.logs.txt 1> all.stats.txt")
        print()
        
    for i, features in enumerate(cfgs):
    
        if create_script:
            print(f"C={i}")
            print("FEATURES=\"", end="")
            
        print("", end="")
        
        has_peep = False
        has_remo_item = False
        has_codemotion = False
        modify_custom_section = False
        
        for name, f in features:
            if "peep" in name:
                has_peep = True
            if "remove_item" in name:
                has_remo_item = True
            if "codemotion" in name:
                has_codemotion = True
            if "code_motion" in name:
                has_codemotion = True
            if "modify_custom_section" in name:
                modify_custom_section = True
            
            print(f"wasm-mutate/{f}", end=",")
        if has_peep:
            print(f"wasm-mutate/peep_hole,", end="")
        if has_remo_item:
            print(f"wasm-mutate/remove_item,", end="")
        if has_codemotion:
            print(f"wasm-mutate/code_motion,", end="")
        if modify_custom_section:
            print(f"wasm-mutate/modify_custom_section,", end="")
        print("\"")
        
        if create_script:
            c = int(attempts/len(cfgs))
            print("cargo build --release --features $FEATURES")
            print(f"RUST_LOG=debug ./target/release/analyzer --dbconn datas/cf$C mutate -s 100 -e --attempts {c}  --input $INPUT --oracle $@ 2>$C.logs.txt 1> $C.stats.txt")
            print()
        
        
        
def print_to_compile_for_argo(cfgs, create_script=True, attempts=1000, pref=""):
    
    for i, features in enumerate(cfgs):
    
        S = ""
        
        has_peep = False
        has_remo_item = False
        has_codemotion = False
        modify_custom_section = False
        
        for name, f in features:
            if "peep" in name:
                has_peep = True
            if "remove_item" in name:
                has_remo_item = True
            if "codemotion" in name:
                has_codemotion = True
            if "code_motion" in name:
                has_codemotion = True
            if "modify_custom_section" in name:
                modify_custom_section = True
            
            S += f"wasm-mutate/{f},"
        if has_peep:
            S += f"wasm-mutate/peep_hole,"
        if has_remo_item:
            S += f"wasm-mutate/remove_item,"
        if has_codemotion:
            S += f"wasm-mutate/code_motion,"
        if modify_custom_section:
            S += f"wasm-mutate/modify_custom_section,"
        
        if create_script:
            #print("cargo build --release --features $FEATURES")
            #print(f"RUST_LOG=debug ./target/release/analyzer --dbconn datas/cf$C mutate -s 100 -e --attempts {c}  --input $INPUT --oracle $@ 2>$C.logs.txt 1> $C.stats.txt")
            print(f"- {{ dbconn: \"c{i}{pref}\", features: \"{S[:-1]}\" }}")
            
        
        

In [23]:
random.seed(10)
c2 = get_uniform_x_filter_non_probable(alldf, 1024, th=0.95, force_included_features=['remove_item_function', 
                                                                                     'remove_item_memory', 
                                                                                     'remove_item_global', 
                                                                                     'remove_item_table'])

print_to_compile_for_argo(c2, attempts=2000)

Set size 21
- { dbconn: "c0", features: "wasm-mutate/i32.shr_s_by_0,wasm-mutate/peep_hole" }
- { dbconn: "c1", features: "wasm-mutate/container-x-drop-i32.rand,wasm-mutate/peep_hole" }
- { dbconn: "c2", features: "wasm-mutate/remove_item_table,wasm-mutate/remove_item" }
- { dbconn: "c3", features: "wasm-mutate/i32.sub_zero,wasm-mutate/remove_item_global,wasm-mutate/peep_hole,wasm-mutate/remove_item" }
- { dbconn: "c4", features: "wasm-mutate/remove_item_memory,wasm-mutate/remove_item" }
- { dbconn: "c5", features: "wasm-mutate/remove_item_function,wasm-mutate/remove_item" }
- { dbconn: "c6", features: "wasm-mutate/add_function" }
- { dbconn: "c7", features: "wasm-mutate/i32.operator_and_commutative,wasm-mutate/peep_hole" }
- { dbconn: "c8", features: "wasm-mutate/i32.add_0,wasm-mutate/i32.sub_zero,wasm-mutate/peep_hole" }
- { dbconn: "c9", features: "wasm-mutate/i32.use_of_global,wasm-mutate/peep_hole" }
- { dbconn: "c10", features: "wasm-mutate/remove_item_global,wasm-mutate/remove_it

In [14]:
import random
random.seed(10)

c2 = get_x_weight_by_applicability(alldf, 1000, reverse=False)

print_to_compile_for_argo(c2)

{'peep.i32.operator_or_neg1': 0.0020092187684670844, 'peep.i32.operator_or_neg_one1': 0.0, 'peep.i32.operator_or_commutative': 0.9888902021037702, 'peep.i64.operator_or_commutative': 0.882637986053658, 'peep.i32.operator_and_commutative': 0.9893629594610566, 'peep.i64.operator_and_commutative': 0.8819288500177284, 'peep.select_same_branches': 0.004018437536934169, 'peep.i32.sub_zero': 0.9891265807824134, 'peep.i32.mul_1': 0.22987826498049876, 'peep.i64.mul_1': 0.8801560099279045, 'peep.f32.mul_1': 0.2097860772958279, 'peep.f64.mul_1': 0.48847653941614466, 'peep.i32.add_0': 0.9881810660678407, 'peep.i64.add_0': 0.8801560099279045, 'peep.f32.add_0': 0.21439546152937006, 'peep.f64.add_0': 0.4897766221486822, 'peep.i32.xor_0': 0.989835716818343, 'peep.i64.xor_0': 0.88204703935705, 'peep.i32.eq_0': 0.9085214513650869, 'peep.i64.eq_0': 0.47110270653587044, 'peep.i32.shl_by_0': 0.9888902021037702, 'peep.i64.shl_by_0': 0.8807469566245125, 'peep.i32.shr_u_by_0': 0.9886538234251271, 'peep.i64.sh

In [27]:
import random
random.seed(10)

c2 = get_all_simple(alldf,reverse=False)

print_to_compile_for_argo(c2, pref="_simple")

- { dbconn: "c0_simple", features: "wasm-mutate/i32.operator_or_neg1,wasm-mutate/peep_hole" }
- { dbconn: "c1_simple", features: "wasm-mutate/i32.operator_or_commutative,wasm-mutate/peep_hole" }
- { dbconn: "c2_simple", features: "wasm-mutate/i64.operator_or_commutative,wasm-mutate/peep_hole" }
- { dbconn: "c3_simple", features: "wasm-mutate/i32.operator_and_commutative,wasm-mutate/peep_hole" }
- { dbconn: "c4_simple", features: "wasm-mutate/i64.operator_and_commutative,wasm-mutate/peep_hole" }
- { dbconn: "c5_simple", features: "wasm-mutate/select_same_branches,wasm-mutate/peep_hole" }
- { dbconn: "c6_simple", features: "wasm-mutate/i32.sub_zero,wasm-mutate/peep_hole" }
- { dbconn: "c7_simple", features: "wasm-mutate/i32.mul_1_t5.snapshot,wasm-mutate/peep_hole" }
- { dbconn: "c8_simple", features: "wasm-mutate/i64.mul_1,wasm-mutate/peep_hole" }
- { dbconn: "c9_simple", features: "wasm-mutate/f32.mul_1,wasm-mutate/peep_hole" }
- { dbconn: "c10_simple", features: "wasm-mutate/f64.mul_1,